In [1]:
import os
import sys


sys.path.append(os.path.abspath(os.path.join(os.path.dirname("__file__"), "src")))

In [2]:
import torch


env_name = "button-press-topdown-v2"
exp_name = "CUDA-test"
pair_algo = "ternary-500"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

TRAJECTORY_LENGTH = 25


In [ ]:
from policy_learning.change_reward_pt import change_reward_and_save_pt
from reward_learning.train_model import train_reward_model

is_linear = True
if is_linear:
    reward_model_algo="PT-linear"
else:
    reward_model_algo="PT-exp"

train_reward_model(
    env_name=env_name,
    exp_name=exp_name,
    pair_algo=pair_algo,
    reward_model_algo=reward_model_algo,
    reward_model_tag="00",
    num_epoch=200,
)

change_reward_and_save_pt(
    env_name=env_name,
    exp_name=exp_name,
    pair_algo=pair_algo,
    is_linear=is_linear,
)

Loaded pair/button-press-topdown-v2/CUDA-test/train/ternary-500.npz dataset
Processed data with 500 samples
Loaded pair/button-press-topdown-v2/CUDA-test/val/ternary-500.npz dataset
Processed data with 500 samples
obs_dim: 39 act_dim: 4
[Train started] reward_model_path: model/button-press-topdown-v2/CUDA-test/reward/ternary-500/PT-linear_00.pth


Training PT reward:   0%|          | 0/200 [00:00<?, ?it/s]